<a href="https://colab.research.google.com/github/Redislabs-Solution-Architects/financial-vss/blob/main/RedisPython_VSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector Search with RedisPy
![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)

This notebook uses [RedisPy](https://redis-py.readthedocs.io/en/stable/), the standard Redis Python client library to perform document + embedding indexing and semantic search tasks.

## Setup and Data Prep

### Pull Github Materials
We need to clone the supporting materials from github.

In [1]:
# This clones your git repository into a directory named 'temp_repo'.
!git clone https://github.com/Redislabs-Solution-Architects/financial-vss.git temp_repo

# This command moves the 'resources' directory from 'temp_repo' to your current directory.
!mv temp_repo/resources .

# This deletes the 'temp_repo' directory, cleaning up the unwanted files.
!rm -rf temp_repo


Cloning into 'temp_repo'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 78 (delta 32), reused 61 (delta 19), pack-reused 0
Receiving objects: 100% (78/78), 6.98 MiB | 13.45 MiB/s, done.
Resolving deltas: 100% (32/32), done.


### Install Python Dependencies

In [2]:
!pip install -q redis "redisvl>==0.0.4" langchain pdf2image "unstructured[all-docs]" sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

### Preprocess PDF Doc(s)

Now we will load a single financial (10k filings) doc and preprocess it using some LangChain helpers.

In [3]:
import os

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader

# Load list of pdfs
data_path = "resources/"
docs = [os.path.join(data_path, file) for file in os.listdir(data_path)]

print("Listing available documents ...", docs)

# For simplicity, we will just work with one of the 10k files. This will take some time still.
# To Note: the UnstructuredFileLoader is not the only document loader type that LangChain provides
# To Note: the RecursiveCharacterTextSplitter is what we use to create smaller chunks of text from the doc.
# Docs: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file
# Docs: https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter
doc = [doc for doc in docs if "nke" in doc][0]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100, add_start_index=True)
loader = UnstructuredFileLoader(doc, mode="single", strategy="fast")
chunks = loader.load_and_split(text_splitter)

print("Done preprocessing. Created", len(chunks), "chunks of the original pdf", doc)

Listing available documents ... ['resources/nvd-10k-2023.pdf', 'resources/nke-10k-2023.pdf', 'resources/msft-10k-2023.pdf', 'resources/amzn-10k-2023.pdf', 'resources/jnj-10k-2023.pdf', 'resources/aapl-10k-2023.pdf']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Done preprocessing. Created 323 chunks of the original pdf resources/nke-10k-2023.pdf


In [4]:
# Take a look at one item
print(chunks[2])

page_content="NIKE, Inc.(Exact name of Registrant as specified in its charter)Oregon93-0584541(State or other jurisdiction of incorporation)(IRS Employer Identification No.)One Bowerman Drive, Beaverton, Oregon 97005-6453(Address of principal executive offices and zip code)(503) 671-6453(Registrant's telephone number, including area code)SECURITIES REGISTERED PURSUANT TO SECTION 12(B) OF THE ACT:Class B Common StockNKENew York Stock Exchange(Title of each class)(Trading symbol)(Name of each exchange on which registered)SECURITIES REGISTERED PURSUANT TO SECTION 12(G) OF THE ACT:NONE\n\nAs of November 30, 2022, the aggregate market values of the Registrant's Common Stock held by non-affiliates were:Class A$7,831,564,572 Class B136,467,702,472 $144,299,267,044\n\nTable of ContentsUNITED STATESSECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549FORM 10-K(Mark One)☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934FOR THE FISCAL YEAR ENDED MAY 31, 2023

### Create document chunk embeddings

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [8]:
%%time
chunk_embeddings = model.encode([chunk.page_content for chunk in chunks])
len(chunk_embeddings) == len(chunks)

CPU times: user 3.33 s, sys: 1.08 s, total: 4.42 s
Wall time: 10.1 s


True

Helper functions to encode the single query vector and display redis search results

In [9]:
import pandas as pd
def encode_one(input):
  return model.encode(input).astype(np.float32).tobytes()

def redis_search_display(res):
  res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
  display(res_df)


### Install Redis Stack (OPTIONAL)

Redis Search will be used as Vector Search engine for LangChain.

Instead of using in-notebook Redis Stack https://redis.io/docs/getting-started/install-stack/ you can provision your own free instance of Redis in the cloud. Get your own Free Redis Cloud instance at https://redis.com/try-free/

In [10]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


### Connect to Redis

By default this notebook would connect to the local instance of Redis Stack. If you have your own Redis Cloud instance - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [11]:
# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

#shortcut for redis-cli $REDIS_CONN command
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"


## VS with Redis Python

### Create the HASH index from schema
Below we connect to Redis and create an index for vector search that contains a single text field and vector field.

In [12]:
import redis
from redis.commands.search.field import TextField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query


r = redis.Redis.from_url(REDIS_URL)

index_name = "redispy"
key_prefix = "doc:redispy"

def create_index(index_type: str = "FLAT"):       # Creates a FLAT index by default
    try:
        # check to see if index exists
        r.ft(index_name).info()
        print("Index already exists!")
    except:
        # schema
        schema = (
            TextField("content"),                  # Text Field Name
            VectorField("chunk_vector",            # Vector Field Name
                index_type, {                      # Vector Index Type: FLAT or HNSW
                    "TYPE": "FLOAT32",
                    "DIM": 384,                    # Number of Vector Dimensions
                    "DISTANCE_METRIC": "COSINE",   # Vector Search Distance Metric
                }
            ),
        )

        # index Definition
        definition = IndexDefinition(prefix=[key_prefix], index_type=IndexType.HASH)   # Uses HASH by default

        # create Index
        r.ft(index_name).create_index(fields=schema, definition=definition)

In [13]:
# Create the index
create_index()

In [14]:
# Check the info related to the newly created index
r.ft(index_name).info()

{'index_name': 'redispy',
 'index_options': [],
 'index_definition': [b'key_type',
  b'HASH',
  b'prefixes',
  [b'doc:redispy'],
  b'default_score',
  b'1'],
 'attributes': [[b'identifier',
   b'content',
   b'attribute',
   b'content',
   b'type',
   b'TEXT',
   b'WEIGHT',
   b'1'],
  [b'identifier',
   b'chunk_vector',
   b'attribute',
   b'chunk_vector',
   b'type',
   b'VECTOR']],
 'num_docs': '0',
 'max_doc_id': '0',
 'num_terms': '0',
 'num_records': '0',
 'inverted_sz_mb': '0',
 'vector_index_sz_mb': '0.00818634033203125',
 'total_inverted_index_blocks': '0',
 'offset_vectors_sz_mb': '0',
 'doc_table_size_mb': '0',
 'sortable_values_size_mb': '0',
 'key_table_size_mb': '0',
 'geoshapes_sz_mb': '0',
 'records_per_doc_avg': 'nan',
 'bytes_per_record_avg': 'nan',
 'offsets_per_term_avg': 'nan',
 'offset_bits_per_record_avg': 'nan',
 'hash_indexing_failures': '0',
 'total_indexing_time': '0',
 'indexing': '0',
 'percent_indexed': '1',
 'number_of_uses': 1,
 'cleaning': 0,
 'gc_stats

### Process and load data using Redis
Below we use a Redis pipeline (not a transaction) to batch send writes to Redis. This method helps with throughput significantly. The batch_size param can be customized and benchmarked on your hardware and with your data. We typically recommend starting small (100-200) and increasing as needed.

In [15]:
# load expects an iterable of dictionaries
import numpy as np

batch_size = 200
pipe = r.pipeline(transaction=False)
for i, chunk in enumerate(chunks):
    data = {
        'content': chunk.page_content,
        # For HASH -- must convert embeddings to bytes
        'chunk_vector': np.array(chunk_embeddings[i]).astype(np.float32).tobytes()
    }
    pipe.hset(f"{key_prefix}:{i}", mapping=data)
    # execute in "mini batches"
    if i % batch_size == 0:
        res = pipe.execute()

# cleanup final batch execution
res = pipe.execute()

In [16]:
# check the data size in Redis
len(chunks) == r.dbsize()

True

In [17]:
# do NOT run this command in production
keys = r.keys()

r.hgetall(keys[0])

{b'chunk_vector': b'\xa0\x13+\xbd\x0b\xe5\x07<!\x19\x8f\xbd\xc3O\xb7\xbc&x\x8c<\x9eI\xb2=\xb0\x08\xe9<\x1c\xa9\xa8=\x06\x0fb=\xb0\x9e\xf1\xbc\xd5\x01_=\xb5\x8bs\xbc\xd6Y\x909l;+\xbd\xb6\xec\xe0\xbcn3O\xbd\x99%\x0c\xbd`T\x86\xbc\xce\xb7\x8e\xbd\xe9\x12\xbf\xbb\xf9\xc1\xfc<O\x99q\xbd\x96$\xa4\xbc\xeb\x91\xa1=!\x88h=\xad\xf1\xb4=r\xc6\xa0\xbbA\xbb\t=|~j\xbc\xc3\xb6\x17\xbd$R\x88\xbdW\x9d\xd7=\x82\x98\x1b=/\xe9\x15\xbc\xea\x81\x93=\x98\x80\x94\xbd<\x91\x06=L\xe5W<2\xfa\x84\xbbE. \xbd\xebO7\xbd\x01\xe8\xfe\xbd\xb0\x1a\xf7<U>\x82:\xfd\xe5*\xba\xd1\xac{=\xdejD\xbdBQ\xcd=d\x92\x1c\xbd\xe4n\xcb=\xab\xee\x03:\x1c\x05\xa9<?R\x86<\x87>\x8e\xbd5\xbb\xfc\xbc\xe4\xee\xb1\xbc\x84\x8e\x02=e\xfbr\xbd!\xae\xa6\xbc\x89\x12x\xbca\xbe\xbd\xb8:g\xbc<7[\x82\xbc\xc2\x8c\x05=\xedA\x16=\xdd$\x81\xbc\xeeH\x9b\xbc\xb4\xaa\x1f\xbd\xee\x9b\xb0\xbd\x9eX\x8e\xba:\xb9\x9b=\xc5-\x85\xbdah\xc8\xbc\x8ad@=bO\x16\xbd\xcb[\xe1;\xae_J=\xec{\xa1\xbbp\x0c&<[\x17\xb1\xbdy\x12\x15=\x8b\x83\xcc=:\xa7\xcb\xba\xcf\xee\xf4\xbc<\xa7B\

### Query the database
Now we can use the Redis search index to perform similarity search operations. This query takes a user input, converts to embeddings, and fetches the top 2 most semantically similar chunks from Redis.

In [18]:
# Grab user input
_input = "Nike profit margins and company performance"

query = (
    Query("*=>[KNN 2 @chunk_vector $vec as score]")
     .sort_by("score")
     .return_fields("content", "score")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vec": encode_one(_input)
}

res=r.ft(index_name).search(query, query_params)
redis_search_display(res)

,id,score,content
0,doc:redispy:150,0.354781925678,2023 FORM 10-K 35\n\nTable of Contents\n\nOPER...
1,doc:redispy:151,0.360561609268,Financial Statements contained in Item 8 of th...


### Range Queries
Range queries allow you to set a pre defined "threshold" for which we want to return documents

In [19]:
query = (
    Query("@chunk_vector:[VECTOR_RANGE $radius $vec]=>{$YIELD_DISTANCE_AS: score}")
     .sort_by("score")
     .return_fields("content", "score")
     .dialect(2)
)

# Find all vectors within 0.8 of the query vector
query_params = {
    "radius": 0.8,
    "vec":encode_one(_input)
}
res=r.ft(index_name).search(query, query_params)
redis_search_display(res)

,id,score,content
0,doc:redispy:150,0.354781925678,2023 FORM 10-K 35\n\nTable of Contents\n\nOPER...
1,doc:redispy:151,0.360561609268,Financial Statements contained in Item 8 of th...
2,doc:redispy:145,0.362202942371,NIKE Brand apparel revenues increased 8% on a ...
3,doc:redispy:219,0.368923187256,BASIS OF CONSOLIDATION\n\nThe Consolidated Fin...
4,doc:redispy:142,0.372902095318,1 %\n\nSales through NIKE Direct Global Brand ...
5,doc:redispy:289,0.376974523067,Asia Pacific & Latin America Global Brand Divi...
6,doc:redispy:204,0.38412541151,To the Board of Directors and Shareholders of ...
7,doc:redispy:287,0.384333252907,Global Brand Divisions is included within the ...
8,doc:redispy:154,0.38654500246,"mix, lower margins in NIKE Direct due to highe..."
9,doc:redispy:153,0.387761950493,"increased 18%, driven by strong digital sales ..."


### Hybrid Queries
Hybrid queries contain both traditional filters (numeric, tags, text) and VS in one single Redis command.

In [20]:
query = (
    Query("@content:profit=>[KNN 2 @chunk_vector $vec as score]")
     .sort_by("score")
     .return_fields("content", "score")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vec": encode_one(_input)
}
res=r.ft(index_name).search(query, query_params)
redis_search_display(res)

,id,score,content
0,doc:redispy:145,0.362202942371,NIKE Brand apparel revenues increased 8% on a ...
1,doc:redispy:140,0.389455199242,COMPARABLE STORE SALES Comparable store sales:...


In [21]:
#r.ft(index_name).dropindex(True)

### What about JSON Support?

Redis also allows you to store data in JSON objects. The JSON fields can contain metadata and vectors. Below is a simple example of indexing JSON data.

**For now** -- JSON support is only enabled in the base redis-py library. It is coming soon to LangChain and RedisVL.

In [22]:
index_name = "redispy:json"
key_prefix = "doc:redispy:json"

# schema
schema = (
    TextField("$.content",                     # Text Field Name (JSON path)
        as_name="content"                      # Text Field Alias -- required for JSON
    ),
    VectorField("$.chunk_vector",              # Vector Field Name (JSON path)
        "FLAT", {                              # Vector Index Type: FLAT or HNSW
            "TYPE": "FLOAT32",
            "DIM": 384,                        # Number of Vector Dimensions
            "DISTANCE_METRIC": "COSINE",       # Vector Search Distance Metric
        },
        as_name="chunk_vector"                 # Vector Field Alias -- required for JSON
    ),
)

# index Definition
definition = IndexDefinition(prefix=[key_prefix], index_type=IndexType.JSON) # select JSON here

# create Index
r.ft(index_name).create_index(fields=schema, definition=definition)

b'OK'

In [23]:
r.ft(index_name).info()

{'index_name': 'redispy:json',
 'index_options': [],
 'index_definition': [b'key_type',
  b'JSON',
  b'prefixes',
  [b'doc:redispy:json'],
  b'default_score',
  b'1'],
 'attributes': [[b'identifier',
   b'$.content',
   b'attribute',
   b'content',
   b'type',
   b'TEXT',
   b'WEIGHT',
   b'1'],
  [b'identifier',
   b'$.chunk_vector',
   b'attribute',
   b'chunk_vector',
   b'type',
   b'VECTOR']],
 'num_docs': '0',
 'max_doc_id': '0',
 'num_terms': '0',
 'num_records': '0',
 'inverted_sz_mb': '0',
 'vector_index_sz_mb': '0.00818634033203125',
 'total_inverted_index_blocks': '7372',
 'offset_vectors_sz_mb': '0',
 'doc_table_size_mb': '0',
 'sortable_values_size_mb': '0',
 'key_table_size_mb': '0',
 'geoshapes_sz_mb': '0',
 'records_per_doc_avg': 'nan',
 'bytes_per_record_avg': 'nan',
 'offsets_per_term_avg': 'nan',
 'offset_bits_per_record_avg': 'nan',
 'hash_indexing_failures': '0',
 'total_indexing_time': '0',
 'indexing': '0',
 'percent_indexed': '1',
 'number_of_uses': 1,
 'cleanin

In [24]:
# Write JSON data to the index

batch_size = 200
pipe = r.pipeline(transaction=False)

for i, chunk in enumerate(chunks):
    redis_key = f"{key_prefix}:{i}"
    data = {
        'content': chunk.page_content,
        'chunk_vector': chunk_embeddings[i].tolist() # notice that we don't need to convert JSON embeddings to bytes
    }
    #print(data)
    pipe.json().set(redis_key, "$", data)
    # mini batch
    if i % batch_size == 0:
        res = pipe.execute()

res = pipe.execute() # make sure to use mini batches if working with larger datasets

In [25]:
# Fetch the JSON doc
r.json().get(f"{key_prefix}:0", "$")

[{'content': 'Indicate by check mark:YESNO•if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act.þ¨•if the registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act.¨þ•whether the registrant (1) has filed all reports required to be filed by Section 13 or 15(d) of the Securities Exchange Act of 1934 during the preceding12 months (or for such shorter period that the registrant was required to file such reports), and (2) has been subject to such filing requirements for thepast 90 days.þ¨•whether the registrant has submitted electronically every Interactive Data File required to be submitted pursuant to Rule 405 of Regulation S-T(§232.405 of this chapter) during the preceding 12 months (or for such shorter period that the registrant was required to submit such files).þ¨•whether the registrant is a large accelerated filer, an accelerated filer, a non-accelerated filer, a smaller reporting company or an emerging gr

In [26]:
# And now you can perform the same kinds of queries

query = (
    Query("@content:profit=>[KNN 2 @chunk_vector $vec as score]")
     .sort_by("score")
     .return_fields("content", "score")
     .paging(0, 2)
     .dialect(2)
)

query_params = {
    "vec": encode_one(_input)

}
res=r.ft(index_name).search(query, query_params)
redis_search_display(res)

,id,score,content
0,doc:redispy:json:145,0.362202942371,NIKE Brand apparel revenues increased 8% on a ...
1,doc:redispy:json:140,0.389455199242,COMPARABLE STORE SALES Comparable store sales:...


## Cleanup
Clean up the index and data.

In [27]:
#r.ft(index_name).dropindex(True)

## What's Next?

Now that you have the basics down with the baseline Redis Python client,

**RedisVL_VS** is probably the next one to try. RedisVL makes it incredibly easy to get started with Redis as a vector database.
<a href="https://colab.research.google.com/github/Redislabs-Solution-Architects/financial-vss/blob/main/RedisVL_VSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LangChain_VS** is next after that -- which will make it easy to integrate Redis with an LLM.
<a href="https://colab.research.google.com/github/Redislabs-Solution-Architects/financial-vss/blob/main/LangChain_VSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>